# Esercitazione 3

In [2]:
def logistic(z):
    return 1/(1+m.e**(-z))

def log_loss(X, y, a, a0, eta=0):
    ll_sum = 0
    for i in range(X.shape[1]):
        x = X[:,i:i+1]
        guess = logistic(a.T.dot(x)[0][0]+a0)
        if guess in (0,1):
            print('*********', guess)
        ll_sum += y[i]*m.log10(guess) + (1-y[i])*m.log10(1-guess)
    return - ll_sum  + eta*(a**2).sum()  # al valore calcolato sommiamo il regolarizzatore

def logistic_der(X, y, a, a0, delta=0.001, eta=0.0):
    g = np.zeros(a.shape)
    for i in range(a.shape[0]):
        delta_v = np.zeros(a.shape)
        delta_v[i,0] = delta
        g[i, 0] = ( log_loss( X, y, a + delta_v, a0, eta) -\
                   log_loss( X, y, a - delta_v, a0, eta) )\
            / (2*delta)
    g0 = ( log_loss( X, y, a, a0+delta, eta) -\
          log_loss( X, y, a, a0-delta, eta) ) / (2*delta)
    return g, g0

def logistic_dg( X, y, step, eps, max_iter, eta=0):
    curr_a = np.zeros( (X.shape[0], 1) )
    curr_a0 = 0
    
    for t in range(max_iter):
        curr_grad, curr_grad0 = logistic_der(X, y, curr_a, curr_a0, eta=eta)
        
        next_a  = curr_a - step*curr_grad
        next_a0 = curr_a0 - step*curr_grad0
        
        if abs( log_loss(X, y, next_a, next_a0, eta=eta) -\
               log_loss(X, y, curr_a, curr_a0, eta=eta) ) < eps:
            curr_a, curr_a0 = next_a, next_a0
            break
        
        curr_a, curr_a0 = next_a, next_a0
    print('iter:', t)
    return curr_a, curr_a0

## Esercizio 1

Tenendo conto che se

$$
f(z) = y\ln \frac{1}{1+e^{-z}} + (1-y)\ln ( 1 -  \frac{1}{1+e^{-z}} )
$$

allora

$$
f'(z) = y-\frac{1}{1+e^{-z}}
$$

Si modifichi la funzione `logistic_der` in modo che utilizzi $f'$ per calcolare il vettore delle derivate parziali al posto del gradiente numerico.


## Esercizio 2 (Discesa del gradiente stocastico)

Modificare le funzioni sopra riportate in modo da implementare l'algoritmo di discesa del gradiente stocastico

## Esercizio 3

Si consideri la shapefile di punti definita dal file `punti.shp`, dopo aver importato il file in un GeoDataFrame ed averlo convertito in un più conveniente sistema di riferimento, si implementi una funzione che preso il GeoDataFrame ed un `float` `eps` ritorni il punto del DataFrame nel cui intorno di raggio `eps` ricadono più punti.